In [32]:
import re
from dotenv import load_dotenv
from dotenv import dotenv_values
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs3 import BeautifulSoup

In [3]:
load_dotenv()
dotenv = dotenv_values()
USERNAME = dotenv.get('USERNAME')
PASSWORD = dotenv.get('PASSWORD')

In [34]:
# Web Driver
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = '1920,1080'

driver_options = Options()
# driver_options.add_argument('--headless')
driver_options.add_argument(f"--window-size={WINDOW_SIZE}")

browser = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                           options=driver_options)

url = "https://kursusvmlepkom.gunadarma.ac.id/login/index.php"
browser.get(url)


In [35]:
username = browser.find_element_by_xpath(
            "//*[@id='username']").send_keys(USERNAME)  # Username Lepkom
password = browser.find_element_by_xpath(
    "//*[@id='password']").send_keys(PASSWORD)  # Password Lepkom
enter_login = browser.find_element_by_xpath(
    "//*[@id='loginbtn']").send_keys(Keys.ENTER)  # Enter

In [73]:
def get_course_list(browser):
    """Get list of all courses."""
    courses = browser.find_elements(By.CLASS_NAME, "coursebox")
    course_names = {
        course.find_element_by_tag_name("a").text:
        course.find_element_by_tag_name("a")
        for course in courses
        if not course.find_element_by_tag_name("a").text.startswith("ACTIVITY")
    }
    return course_names
course_names = get_course_list(browser=browser)
course_names

{'DBMS Testing': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="f23b11e6-b534-44b3-9256-6ac7f39d3da8")>,
 '1BWIR181153K - WINDOWS SERVER FOR BEGINNER': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="3d108ab2-2521-4d7f-9622-af9ef5720d55")>,
 '1IGGR181250K - GOLANG FOR INTERMEDIATE': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="ea75395e-52f8-4334-84c3-bf40e4b86367")>,
 '1FWER181442K - FUNDAMENTAL WEB': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="28b4dee6-97f7-4830-9b3a-89f18880e033")>,
 '1IGGR181611K - GOLANG FOR INTERMEDIATE': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="0629093a-bc79-47b5-905d-6eb917ce8678")>,
 '3BGGR171029L - GOLANG FOR BEGINNER (2KA27)': <selenium.webdriver.remote.webelement.WebElement

In [74]:
course = course_names['3BGGR171029L - GOLANG FOR BEGINNER (2KA27)']
# browser.send_keys(course)
course.send_keys(Keys.ENTER)

In [18]:
browser.find_element_by_id("section-2")

<selenium.webdriver.remote.webelement.WebElement (session="a99864d79c85a06d65ae33b9cf43ab08", element="58055da3-f3ac-4359-9d61-778744cb2a29")>

In [71]:
browser.back()

In [109]:
def get_pertemuan(browser):
    sections = browser.find_elements_by_css_selector("[id^='section']")
    sections = [section for section in sections if section.text.lower().startswith(("pertemuan", "ujian"))]
    return sections

def get_quiz(pertemuan):
    instances = pertemuan.find_elements_by_class_name("instancename")
    buttons = []
    regex = re.compile(r'([\w\s]+)\n')
    for instance in instances:
        button_name = instance.text
        button_name = regex.search(button_name).group(1)
        button = (button_name, instance)
        buttons.append(button)
    
    return buttons

def get_pertemuan_quiz(browser):
    list_pertemuan = get_pertemuan(browser)
    quiz_dan_pertemuan = {}

    for idx, pertemuan in enumerate(list_pertemuan,1):
        quiz_dan_pertemuan['Pertemuan '+str(idx)] = {}
        for nama_quiz, element in get_quiz(pertemuan):
            quiz_dan_pertemuan['Pertemuan '+str(idx)][nama_quiz] = element

    return quiz_dan_pertemuan


cek = get_pertemuan_quiz(browser)
cek

{'Pertemuan 1': {'Pre Test 1': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="41573d7f-9600-474a-83a9-c4e3943198c8")>,
  'Post Test 1': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="bbe24113-8d04-4465-8868-b72c9b40baa3")>,
  'Upload Activity Pertemuan 1': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="bf165034-4145-4fb7-9b62-6eb0bff0a2aa")>,
  'Materi Minggu 1': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="5a68ba2a-b3cf-459c-8494-8dfb7e76d262")>},
 'Pertemuan 2': {'Pre Test 2': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="60dd1e37-b3e3-452e-b695-3ddc7fac44a5")>,
  'Post Test 2': <selenium.webdriver.remote.webelement.WebElement (session="55b531d2db69f958b107a28934ee9c2f", element="328f7b06-1055-4787-9b7c-90

In [63]:

section = browser.find_element_by_id("section-2").find_elements_by_class_name("instancename")
# actions = ActionChains(browser)

# actions.move_to_element(section)
# actions.click(section)
# actions.perform()
[instance.text for instance in section]

['Pre Test 1\nQuiz',
 'Post Test 1\nQuiz',
 'Upload Activity\nAssignment',
 'Materi Minggu 1\nURL']